In [36]:
import pandas as pd
import numpy as np
import seaborn as sn
import tensorflow as tf
from numbers import Number
from time import time
from ta import momentum as mo
from ta import volume as vo
from ta import trend as tr
from numpy.random import seed
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout, Activation, Flatten, GRU, LSTM
from keras.utils import np_utils
from keras.callbacks import CSVLogger, ModelCheckpoint
# seed(1)
# tf.random.set_seed(2)

In [3]:
year = 31536000
period1 = round(time() - (year * 5))
period2 = round(time())
asset = 'qdel'

hist_data = pd.read_csv(
    f'https://query1.finance.yahoo.com/v7/finance/download/{asset}?period1={period1}&period2={period2}&interval=1d&events=history&includeAdjustedClose=true')

In [4]:
# ROC de Y em relação a X
def change(x, y):
    if len(x) == len(y):
        change_arr = []
        for i in range(0, len(x)):
            if isinstance(x[i], Number) and isinstance(y[i], Number):
                change_arr.append(((y[i] - x[i]) / x[i]) * 100)
            else:
                change_arr.append('NaN')

        return change_arr
    else:
        print(f"Error: x length {len(x)}, y length {len(y)}")

In [5]:
# print(change([1, 2],[2, 1])) # [100.0, -50.0] Yn > Xn se valor positivo
trend_window = 26
slice_start = 50
slice_plus = 0

roc = mo.roc(hist_data['Close'], window=25)
rsi = mo.rsi(hist_data['Close'])
stochrsi = mo.stochrsi(hist_data['Close'])
stochrsi_d = mo.stochrsi_d(hist_data['Close'])
stochrsi_k = mo.stochrsi_k(hist_data['Close'])

tsi = mo.tsi(hist_data['Close'])
tsi_signal = tr.ema_indicator(tsi, window=13)

macd = tr.macd(hist_data['Close'])
macd_diff = tr.macd_diff(hist_data['Close'])
macd_signal = tr.macd_signal(hist_data['Close'])

stoch = mo.stoch(hist_data['High'], hist_data['Low'], hist_data['Close'])
ema_w8 = tr.ema_indicator(hist_data['Close'], window=8)
ema_w20 = tr.ema_indicator(hist_data['Close'], window=20)

sma_w13 = tr.sma_indicator(hist_data['Close'], window=13)
sma_w25 = tr.sma_indicator(hist_data['Close'], window=25)

roc_close_ema_w8 = change(ema_w8, hist_data['Close'])
roc_ema_w8_ema_w20 = change(ema_w20, ema_w8)
roc_close_ema_w20 = change(ema_w20, hist_data['Close'])
roc_close_open = change(hist_data['Open'], hist_data['Close'])

roc_close_sma_w13 = change(sma_w13, hist_data['Close'])
roc_sma_w13_sma_w25 = change(sma_w25, sma_w13)
roc_close_sma_w25 = change(sma_w25, hist_data['Close'])

trend = mo.roc(hist_data['Close'], window=trend_window).map(lambda n : 'HIGH' if n > 0 else 'LOW')

df = pd.DataFrame({
#     'close': np.array(list(hist_data['Close'][slice_start:-(trend_window + slice_plus)])),
    'roc': np.array(list(roc[slice_start:-(trend_window + slice_plus)])),
    'rsi': np.array(list(rsi[slice_start:-(trend_window + slice_plus)])),
    # 'stoch': np.array(list(stoch[slice_start:-(trend_window + slice_plus)])),
    # 'stochrsi': np.array(list(stochrsi[slice_start:-(trend_window + slice_plus)])),
    # 'stochrsi_d': np.array(list(stochrsi_d[slice_start:-(trend_window + slice_plus)])),
    # 'stochrsi_k': np.array(list(stochrsi_k[slice_start:-(trend_window + slice_plus)])),
    'macd': np.array(list(macd[slice_start:-(trend_window + slice_plus)])),
    'macd_diff': np.array(list(macd_diff[slice_start:-(trend_window + slice_plus)])),
    'macd_signal': np.array(list(macd_signal[slice_start:-(trend_window + slice_plus)])),
    'tsi': np.array(list(tsi[slice_start:-(trend_window + slice_plus)])),
    'tsi_signal': np.array(list(tsi_signal[slice_start:-(trend_window + slice_plus)])),
#     'ema_w8': np.array(list(ema_w8[slice_start:-(trend_window + slice_plus)])),
#     'ema_w20': np.array(list(ema_w20[slice_start:-(trend_window + slice_plus)])),
    'roc_close_ema_w8': np.array(list(roc_close_ema_w8[slice_start:-(trend_window + slice_plus)])),
    'roc_ema_w8_ema_w20': np.array(list(roc_ema_w8_ema_w20[slice_start:-(trend_window + slice_plus)])),
#     'roc_close_ema_w20': np.array(list(roc_close_ema_w20[slice_start:-(trend_window + slice_plus)])),

    'roc_close_sma_w13': np.array(list(roc_close_sma_w13[slice_start:-(trend_window + slice_plus)])),
    'roc_sma_w13_sma_w25': np.array(list(roc_sma_w13_sma_w25[slice_start:-(trend_window + slice_plus)])),
#     'roc_close_sma_w25': np.array(list(roc_close_sma_w25[slice_start:-(trend_window + slice_plus)])),
    'trend': np.array(list(trend[(slice_start + trend_window)::]))
})

df

,roc,rsi,macd,macd_diff,macd_signal,tsi,tsi_signal,roc_ema_w8_ema_w20,roc_sma_w13_sma_w25,trend
0,21.938226,58.789981,0.360335,-0.012137,0.372472,11.821174,11.903428,1.400998,2.125317,LOW
1,15.902135,56.673748,0.350624,-0.017479,0.368103,11.817746,11.891187,1.333074,1.336719,LOW
2,15.607240,52.011209,0.309504,-0.046879,0.356383,10.511356,11.694069,1.048557,0.393053,LOW
3,7.864073,50.207805,0.261795,-0.075671,0.337465,8.919609,11.297717,0.741426,-0.118466,LOW
4,-1.774867,38.733700,0.126503,-0.168769,0.295273,3.573473,10.194254,-0.168199,-0.598476,LOW
...,...,...,...,...,...,...,...,...,...,...
1176,-1.911143,39.177892,-1.095657,-2.946468,1.850811,-3.525519,10.601664,-2.786292,-0.136326,HIGH
1177,2.478571,39.961439,-1.717034,-2.854276,1.137242,-5.952244,8.236820,-3.175945,-1.501639,HIGH
1178,1.688351,37.734455,-2.398089,-2.828265,0.430176,-8.666748,5.822025,-3.662141,-2.905678,HIGH
1179,-2.106705,34.818330,-3.202699,-2.906300,-0.296399,-11.884818,3.292476,-4.322194,-4.321340,HIGH


In [6]:
x_train = df.iloc[:,0:-1].values[0:-100]
y_train = df.iloc[:,-1].map({'LOW': 0, 'HIGH': 1}).values[0:-100]

x_test = df.iloc[:,0:-1].values[-100::]
y_test = df.iloc[:,-1].map({'LOW': 0, 'HIGH': 1}).values[-100::]

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

step_size = x_train.shape[1]
nb_features = x_train.shape[2]
x_train.shape

(1081, 9, 1)

In [43]:
model = Sequential()
model.add(Conv1D(20, 8, activation="relu", input_shape=(9,1)))
model.add(Dense(15, activation="relu"))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(2, activation = 'softmax'))
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
model.summary()
model.fit(x_train, y_train, batch_size=16, epochs=100, validation_data=(x_test,y_test))


# CNN
# model = Sequential()
# model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=3, filters=20, kernel_size=4))
# model.add(Dropout(0.5))
# model.add(Conv1D(strides=4, filters=nb_features, kernel_size=2))

# model.compile(loss='mse', optimizer='adam', metrics = ['accuracy'])
# model.fit(x_train, y_train, validation_data=(x_test,y_test), batch_size=8, epochs=100)

# LSTM
# model = Sequential()
# model.add(LSTM(units=100,activation='relu', input_shape=(step_size,nb_features),return_sequences=False))
# model.add(Dropout(0.2))
# model.add(Dense(2))
# model.add(LeakyReLU())
# model.compile(loss='mse', optimizer='adam')
# model.fit(x_train, y_train, batch_size=2,validation_data=(x_test,y_test), epochs = 100)

# GRU
# model = Sequential()
# model.add(GRU(units=50, input_shape=(step_size,nb_features),return_sequences=False))
# model.add(Activation('tanh'))
# model.add(Dropout(0.2))
# model.add(Dense(2))
# model.add(Activation('relu'))
# model.compile(loss='mse', optimizer='adam', metrics = ['accuracy'])
# model.fit(x_train, y_train, batch_size=8,validation_data=(x_test,y_test), epochs = 100)

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_29 (Conv1D)           (None, 2, 20)             180       
_________________________________________________________________
dense_33 (Dense)             (None, 2, 15)             315       
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 1, 15)             0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 15)                0         
_________________________________________________________________
dense_34 (Dense)             (None, 2)                 32        
Total params: 527
Trainable params: 527
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
68/68 [==============================] - 1s 4ms/step - loss: 1.6457 - accuracy: 0.5199 - val_loss: 0.6772 - v